In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv("temp_data.csv")

In [3]:
print(df.shape)
print(df.isna().sum())
print(df.columns)

(1235, 27)
SentAt             0
StartedAt        691
Control          691
Sad              691
Intrusion        691
Encourage        691
FunThings        691
IgotThis         691
Company          691
Alone            691
Thoughts         691
Safe             691
Appointments     691
Cancel           691
Laying           691
Sleep            691
Qualitysleep    1072
Out              691
Eaten            691
Avoid            691
Useful           691
Enjoy            691
Pleas            691
Pleasant        1076
Unpl             691
Unpleasant      1211
Remarks         1087
dtype: int64
Index(['SentAt', 'StartedAt', 'Control', 'Sad', 'Intrusion', 'Encourage',
       'FunThings', 'IgotThis', 'Company', 'Alone', 'Thoughts', 'Safe',
       'Appointments', 'Cancel', 'Laying', 'Sleep', 'Qualitysleep', 'Out',
       'Eaten', 'Avoid', 'Useful ', 'Enjoy', 'Pleas', 'Pleasant', 'Unpl',
       'Unpleasant', 'Remarks'],
      dtype='object')


In [4]:
#make idvar
df['id_var'] = 1
#make date
df['Date'] = pd.to_datetime(df['SentAt'], format = '%d/%m/%Y %H:%M')
#get the date and hour
df['Date_2'] = [d.strftime("%x") for d in df['Date']]
df['Hour'] = [d.strftime("%H") for d in df['Date']]

In [5]:
#high number of nas so we drop them
df = df.drop(columns = ['StartedAt', 'Qualitysleep', 'Pleasant', 'Unpleasant', 'Remarks'])
#note that we still capture pleasant and unpleasant through the binary variables

In [6]:
#therefore we drop all observations (rows) that are incomplete
to_drop =  ['Control', 'Sad', 'Intrusion', 'Encourage',
       'FunThings', 'IgotThis', 'Company', 'Alone', 'Thoughts', 'Safe',
       'Appointments', 'Cancel', 'Laying', 'Sleep', 'Out',
       'Eaten', 'Avoid', 'Useful ', 'Enjoy', 'Pleas', 'Unpl']
#select only rows that have complete data
df = df.dropna(subset = to_drop, how = 'any', axis=0)

In [7]:
##below we create a beep_var - i.e. a unique identifier for the 'beep' for a given day. Required for graphicalvar.
#Groupings are because the timing of survey completion changes during the period examined.
df['first_obs'] = [1 if (x == '09' or x == '10') else 0 for x in df['Hour']]
df['second_obs'] = [2 if (x == '15' or x == '16') else 0 for x in df['Hour']]
df['third_obs'] = [3 if (x == '21' or x == '22') else 0 for x in df['Hour']]
df['beep_var'] = df['first_obs'] + df['second_obs'] + df['third_obs']

#below we create a day_var - i.e. a unique identifier for the day of completion. Required for graphicalvar. 
day_var = pd.DataFrame(pd.date_range(df['Date'].min(), df['Date'].max()))
day_var.columns = ['Date_Range']
day_var['day_var'] = [d.strftime("%x") for d in day_var['Date_Range']]
day_var = day_var.drop('Date_Range', axis = 1).reset_index()
df = df.merge(day_var, how = 'left', left_on = 'Date_2', right_on = 'day_var')


#order by date
df = df.sort_values(by='Date')

In [8]:
#clean by dropping unnecessary columns and renaming columns
df = df.drop(['first_obs', 'second_obs', 'third_obs', 'day_var'], axis = 1)
#rename index to day_var
df = df.rename(columns = {'index': 'day_var'})
#investigate
df['consecutive'] = df['day_var'].diff()
#because R doesn't start from -
df['day_var'] = df['day_var']  + 1
#conver variables to string
#df['day_var'] = df['day_var'].astype(str)
#df['id_var'] = df['id_var'].astype(str)
#df['beep_var'] = df['beep_var'].astype(str)
#print max gap between days
print(df['consecutive'].max())

6.0


In [ ]:
#check if the nas are just empty observations for all variables
na_count = [df.iloc[x, :].isna().sum() for x in range(0, len(df))]
pd.Series(na_count).value_counts()
#yes they are

In [ ]:
df.to_csv('ESM_all.csv')

In [ ]:
# Given the missingness in the data, we simply subselect for the first observation (beep) recorded on a given day
df_first = df.drop_duplicates(subset='day_var', keep='first', inplace=False, ignore_index=False)
df_first['consecutive'] = df_first['day_var'].diff()
#check which beeps we are actually using
df_first['beep_var'].value_counts()

In [ ]:
df_first['beep_var_consecutive'] = df_first['beep_var'].diff()
df_first['beep_var_consecutive'].value_counts()

In [ ]:
df_first.shape

In [ ]:
df_first.to_csv('ESM_first.csv')